In [1]:
import pyam
import pandas as pd
import numpy as np

import dotenv
import os
from pathlib import Path

<IPython.core.display.Javascript object>

In [2]:
dotenv.load_dotenv()

True

Step 1: Read in the global AR6 dataset and downselect for the following potential filters:
* Primary Energy|Biomass
* Land Cover|Cropland|Energy Crops

In [3]:
df = pd.read_csv(
    os.environ['AR6_DATA'],
    index_col=[0,1]
)

In [4]:
df_bio_pe = (
    df[df['Variable']=='Primary Energy|Biomass']
)

In [5]:
df_land_cover = (
    df[df['Variable']=='Land Cover|Cropland|Energy Crops']
)

Step 2: Read in the scenarios that are covered by Gidden et al.

In [6]:
gidden_scens = pd.read_csv(
    os.environ['GIDDEN_META'],
    index_col=[0,1]
)

Step 3: Pull out the intersection of the scenarios.

In [7]:
scens_intersection = (
    df_bio_pe
    .index
    .intersection(gidden_scens.index)
)

In [8]:
df_bio_pe_filtered = (
    df_bio_pe
    .loc[scens_intersection]
)

In [9]:
df_bio_pe_filtered_pyam = pyam.IamDataFrame(
    df_bio_pe_filtered
    .reset_index()
)

In [10]:
scens_intersection_land = (
    df_land_cover
    .index
    .intersection(gidden_scens.index)
)

In [11]:
df_land_cover_filtered = pyam.IamDataFrame(
    df_land_cover
    .loc[scens_intersection_land]
    .reset_index()
)

Step 4: Identify the maximum level of PE bioenergy as well as the maximum land cover allocated to energy crops.

In [12]:
df_bio_pe_filtered_pyam.set_meta_from_data(
    name='pe_bio_max',
    variable='Primary Energy|Biomass',
    method=np.max
)

In [13]:
df_land_cover_filtered.set_meta_from_data(
    name='land_cover_max',
    variable='Land Cover|Cropland|Energy Crops',
    method=np.max
)

Step 5: Set a new exclude column to False.

In [14]:
df_bio_pe_filtered_pyam.set_meta(
    name='bio_sustainable',
    meta=False
)

In [15]:
df_land_cover_filtered.set_meta(
    name='land_sustainable',
    meta=False
)

In [16]:
scens_pass = (
    df_bio_pe_filtered_pyam.meta[df_bio_pe_filtered_pyam.meta['pe_bio_max'] <= 100].index
)

In [17]:
df_bio_pe_filtered_pyam.set_meta(
    name='bio_sustainable',
    meta=True,
    index=scens_pass
)

In [18]:
filter_set_bio = df_bio_pe_filtered_pyam.meta

In [19]:
filter_set_bio.to_csv(
    Path(
        '../data/103_filter_set_bio.csv'
    )
)

In [20]:
df_land_cover_filtered.meta.to_csv(
    Path(
        '../data/103_filter_set_land.csv'
    )
)